https://www.kaggle.com/c/cs5785-fall19-final/

In [7]:
pip install -U gensim

  Stored in directory: C:\Users\evatr\AppData\Local\pip\Cache\wheels\ab\10\93\5cff86f5b721d77edaecc29959b1c60d894be1f66d91407d28
Successfully built smart-open
Note: you may need to restart the kernel to use updated packages.


In [108]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from os import listdir
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, accuracy_score
import csv
import sklearn
import os
import random
import gensim
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import NearestNeighbors
#stopWords = stopwords.words('english')
#isStopWord = lambda w: w in stopWords or len(w) == 1

In [9]:
# load word vectors
num_train = 8000
num_dev = 2000
num_test = 2000
split_idx = list(range(num_train + num_dev))
random.shuffle(split_idx)
word2vec = gensim.models.KeyedVectors.load_word2vec_format("../GoogleNews-vectors-negative300.bin.gz", binary=True)
print("Loaded word vectors successfully!")

#from gensim.models import Word2Vec
#word2vec = Word2Vec.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True, norm_only=True)

Loaded word vectors successfully!


In [168]:
def parse_descriptions(data_dir, num_doc):
    docs = []
    for i in range(num_doc):
        path = os.path.join(data_dir, "%d.txt" % i)
        with open(path) as f:
            docs.append(f.read())
    return docs

def doc_to_vec(sentence, word2vec):
    # get list of word vectors in sentence
    word_vecs = [word2vec.get_vector(w) for w in sentence.split() if w in word2vec.vocab]
    # return average
    return np.stack(word_vecs).mean(0)

def tag_to_vec(sentence, word2vec):
    # get list of word vectors in sentence
    print(sentence)
    word_vecs = [word2vec.get_vector(w) for w in sentence.split(':') if w in word2vec.vocab]
    # return average
    return np.stack(word_vecs).mean(0)

# build x matrices
train_dev_desc = parse_descriptions("cs5785-fall19-final/descriptions_train", num_doc=(num_train+num_dev))
train_dev_tags = parse_descriptions("cs5785-fall19-final/tags_train", num_doc=(num_train+num_dev))
test_desc = parse_descriptions("cs5785-fall19-final/descriptions_test", num_doc=num_test)
x_train = np.array([doc_to_vec(train_dev_desc[i], word2vec) for i in split_idx[:num_train]])
tags_train = np.array([tag_to_vec(train_dev_tags[i], word2vec) for i in split_idx[:num_train]])
x_dev = np.array([doc_to_vec(train_dev_desc[i], word2vec) for i in split_idx[num_train:]])
x_test = np.array([doc_to_vec(d, word2vec) for d in test_desc])

print("Built all x matrices!")
print("x_train shape:", x_train.shape)
print("tags_train shape:", tags_train.shape)
print("x_dev shape:", x_dev.shape)
print("x_test shape:", x_test.shape)

ValueError: need at least one array to stack

In [171]:
print(train_dev_tags)

['vehicle:airplane\noutdoor:bench\nsports:skateboard\nperson:person\nvehicle:truck\naccessory:backpack\naccessory:handbag\nfurniture:dining table\n', 'kitchen:bowl\nfood:carrot\nkitchen:spoon\n', 'vehicle:car\nvehicle:truck\noutdoor:traffic light\nperson:person\n', 'person:person\noutdoor:bench\nsports:frisbee\nvehicle:car\n', 'person:person\nsports:baseball bat\n', 'furniture:bed\nfurniture:chair\nelectronic:mouse\nelectronic:keyboard\nindoor:book\nkitchen:cup\nelectronic:tv\nelectronic:laptop\n', 'person:person\nfood:donut\nvehicle:bicycle\n', 'person:person\naccessory:tie\n', 'vehicle:car\nperson:person\n', 'vehicle:car\nvehicle:bus\naccessory:backpack\n', 'accessory:suitcase\n', 'person:person\nsports:baseball bat\nsports:baseball glove\n', '', 'outdoor:bench\nsports:baseball bat\nsports:baseball glove\nperson:person\n', 'vehicle:car\noutdoor:stop sign\n', 'kitchen:wine glass\nperson:person\nkitchen:bottle\nfurniture:chair\n', 'indoor:clock\nperson:person\naccessory:backpack\nvehic

In [164]:
def parse_features(features_path):
    vec_map = {}
    with open(features_path) as f:
        for row in csv.reader(f):
            img_id = int(row[0].split("/")[1].split(".")[0])
            vec_map[img_id] = np.array([float(x) for x in row[1:]])
    return np.array([v for k, v in sorted(vec_map.items())])

# build y matrices
p = np.random.randn(1000, 100)
y_train_dev = parse_features("cs5785-fall19-final/features_train/features_resnet1000_train.csv") @ p
test_dev_tags = parse_descriptions("cs5785-fall19-final/tags_test", num_doc=(num_train+num_dev))
y_train = y_train_dev[split_idx[:num_train]]
tags_test = np.array([tag_to_vec(train_dev_tags[i], word2vec) for i in split_idx[:num_train]])
y_dev = y_train_dev[split_idx[num_train:]]
y_test = parse_features("cs5785-fall19-final/features_test/features_resnet1000_test.csv") @ p

print("Built all y matrices!")
print("y_train shape:", y_train.shape)
print("tags_test shape:", tags_test.shape)
print("y_dev shape:", y_dev.shape)
print("y_test shape:", y_test.shape)

FileNotFoundError: [Errno 2] No such file or directory: 'cs5785-fall19-final/tags_test\\2000.txt'

In [64]:
# function to train onevsrest logistic regression model, predict with it, and calculate the accuracy and confusion matrix
def Logistic_Regression(x_train, y_train, x_test, y_test):

    # initialize and fit logistic regression model with training data
    #lr = LogisticRegression(solver = 'lbfgs', max_iter = 10000)
    lr = LogisticRegression(multi_class='multinomial', solver='lbfgs')
    lr.fit(x_train, y_train)
    
    # predict the result for the testing data
    lr_pred = lr.predict(x_test) 
    
    # calculate accuracy
    lr_acc = accuracy_score(lr_pred, y_test)
    
    # confusion matrix
    cfn_matrix_lr = confusion_matrix(y_test, lr_pred)
   
    # return accuracy and confusion matrix
    return lr_acc,cfn_matrix_lr

In [14]:
# function to train gaussian naive bayes model, predict with it, and calculate the accuracy and confusion matrix
def Gaussian_NB(x_train, y_train, x_test, y_test):
    
    # initialize and fit naive bayes model gaussian prior with training data
    gnb = GaussianNB()
    gnb.fit(x_train, y_train)

    # predict the results for the test set
    gnb_pred = gnb.predict(x_test) 
    
    # calculate accuracy
    gnb_acc = accuracy_score(gnb_pred, y_test)
    
    # confusion matrix
    cfn_matrix_gnb = confusion_matrix(y_test, gnb_pred)
    
    # return accuracy and confusion matrix
    return gnb_acc, cfn_matrix_gnb

In [83]:
# function to train random forest regressor model, predict with it, and calculate the accuracy and confusion matrix
def Random_Forest(x_train, y_train, x_test, y_test):
    # set up regressor  
    rf_regressor = RandomForestRegressor(max_depth=20) 
    
    # fit regressor 
    rf_regressor.fit(x_train, y_train)
    
    # predict
    rf_preds = rf_regressor.predict(x_test)
    
    # return accuracy and confusion matrix
    return rf_preds

In [147]:
# function to train KNN regressor model, predict with it, and calculate the accuracy and confusion matrix
def KNN_Regressor(x_train, y_train, x_test, y_test):
    
    # set up regressor  
    knn_regressor = KNeighborsRegressor(n_neighbors = 1) 
    
    # fit regressor 
    knn_regressor.fit(x_train, y_train)
    
    # predict
    knn_preds = knn_regressor.predict(x_test)
    
    # return accuracy and confusion matrix
    return knn_preds

In [137]:
# function to return the number of nearest neighbors specified
def knn_function(data, point, neighbors_number):
    knn = NearestNeighbors(n_neighbors=neighbors_number, algorithm='ball_tree')
    knn.fit(data)
    return knn.kneighbors(point, neighbors_number, return_distance = False)

In [151]:
def scores(i):
    return (20+1-(i+1))/20

In [152]:
def find_neighbors_and_eval_accuracy(ytest, ypred):
    # FIND NEAREST NEIGHBORS
    neighbors = knn_function(ytest, ypred, 20)
    # EVALUATE THE MODEL USING THE MEAN AVERAGE PRECISION AT 20
    scs = []
    for i in range(len(ypred)):
        good = False
        for j, n in enumerate(neighbors[i]):
            if i == n:
                scs.append(scores(j))
                good = True
                break
        if good == False:
            scs.append(0)
    return sum(scs)/len(ypred)

In [24]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# train OLS model with regression
parameters = {"alpha": [0.01, 0.05, 0.1, 0.5, 1.0, 5.0, 10.0]}
reg = GridSearchCV(Ridge(), parameters, cv=10)
reg.fit(x_train, y_train)
reg_best = reg.best_estimator_

print("Trained linear regression model!")
print("Summary of best model:")
print(reg_best)

C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').


C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').


C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').


C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').


Trained linear regression model!
Summary of best model:
Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)


C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').


In [35]:
print(x_train.shape)
print(y_train.shape)

(8000, 300)
(8000, 100)


In [65]:
# using sum of all columns as a singular value for label 
lab_enc = preprocessing.LabelEncoder()
new_y_train = np.sum(y_train, axis = 1)
new_y_test = np.sum(y_test, axis = 1)
encoded_new_y_train = lab_enc.fit_transform(new_y_train)
encoded_new_y_test = lab_enc.fit_transform(new_y_test)
lr_acc, cfn_matrix_lr = Logistic_Regression(x_train, encoded_new_y_train, x_test, encoded_new_y_test)

In [66]:
print(lr_acc)

0.0005


In [67]:
gnb_acc, cfn_matrix_gnb = Gaussian_NB(x_train, encoded_new_y_train, x_test, encoded_new_y_test)

In [68]:
print(gnb_acc)

0.0


Based on these accuracies we discard this approach

In [74]:
from scipy import stats

kernel = stats.gaussian_kde(y_train)

print(kernel)

In [148]:
knn_preds = KNN_Regressor(x_train, y_train, x_test, y_test)

-1.0446926011145343


C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


In [149]:
find_neighbors_and_eval_accuracy(y_test, knn_preds)

0.0035250000000000004

In [145]:
def dist(x1, x2):
    return sum((x1 - x2) ** 2) ** 0.5

def evaluate_model_with_error(ytest, ypred):
    # EVALUATE THE MODEL USING SUM SQUARED ERROR
    scores1 = []
    for i in range(len(ypred)):
        distances = []
        for j in range(len(ytest)):
            distances.append(dist(ypred[i], ytest[j]))
        pred_dist_idx = list(np.argsort(distances))
        dev_pos = pred_dist_idx.index(i)
        if dev_pos < 20:
            scores1.append(scores(dev_pos))
        else:
            scores1.append(0.0)
    return np.mean(scores1)

In [150]:
evaluate_model_with_error(y_test, knn_preds)

0.0035250000000000004

That is the accuracy with 1 neighbor for the top 20 images using KNN to select these, but with 20 neighbors it was similar

In [93]:
from scipy.spatial import distance_matrix
dm = distance_matrix(knn_preds, y_test)

In [94]:
id_dists = []
for i, d in enumerate(dm):
    temp_dists = [(j, d[j]) for j in range(len(d))]
    id_dists.append(list(sorted(temp_dists, key= lambda x: abs(x[1])))[:20])

In [95]:
top_images = [["{}.jpg".format(id) for id, _ in row] for row in id_dists]

In [96]:
def outputCSV(predictions):
    with open("image_prediction.csv", "w") as outputFile:
        headers = ["Description_ID", "Top_20_Image_IDs"]
        fileWriter = csv.DictWriter(outputFile, fieldnames=headers)
        fileWriter.writeheader()
        for index, pred in enumerate(predictions):
            fileWriter.writerow({headers[0]: "{}.txt".format(index), headers[1]: ' '.join(predictions[index])})

In [97]:
outputCSV(top_images)

In [ ]:
# Define a pipeline combining a feature extractor with multi label classifier
NB_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))),
            ])
for category in categories:
    print('... Processing {}'.format(category))
    # train the model using X_dtm & y
    NB_pipeline.fit(X_train, train[category])
    # compute the testing accuracy
    prediction = NB_pipeline.predict(X_test)
    print('Test accuracy is {}'.format(accuracy_score(test[category], prediction)))

In [42]:
# Create logistic regression
logistic = LogisticRegression()

# Create regularization penalty space
penalty = ['l1', 'l2']

# Create regularization hyperparameter space
C = np.logspace(0, 4, 10)

# Create hyperparameter options
hyperparameters = dict(C=C, penalty=penalty)

# Create grid search using 5-fold cross validation
clf = GridSearchCV(logistic, hyperparameters, cv=5, verbose=0)

# Fit grid search
best_model = clf.fit(x_train, y_train)

# View best hyperparameters
print('Best Penalty:', best_model.best_estimator_.get_params()['penalty'])
print('Best C:', best_model.best_estimator_.get_params()['C'])

# Predict target vector
best_model.predict(x_test)

C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:528: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


ValueError: bad input shape (6400, 100)

### Not running from here to test the part above

In [99]:
dataFolder = "cs5785-fall19-final"

descTrainFolder = dataFolder + "/descriptions_train"
descTestFolder = dataFolder + "/descriptions_test"

featTrainFolder = dataFolder + "/features_train"
featTestFolder = dataFolder + "/features_test"

imagesTrainFolder = dataFolder + "/images_train"
imagesTestFolder = dataFolder + "/images_test"

tagsTrainFolder = dataFolder + "/tags_train"
tagsTestFolder = dataFolder + "/tags_test"

folders = [descTrainFolder,   descTestFolder,   featTrainFolder, featTestFolder, 
           imagesTrainFolder, imagesTestFolder, tagsTrainFolder, tagsTestFolder]

In [100]:
def getFilesFromFolder(folder):
    return listdir(folder)

In [101]:
# function to preprocess data
def preprocessing(data):
    stop_words = set(stopwords.words('english')) # find stop words in English language
    lemmatizer = WordNetLemmatizer() # declare nltk lemmatizer

    # iterate through every sentence and replace it by itself lemmatized, without punctuation and without stop words
    for i in range(len(data)):
        sentence_no_punct = ''
        # remove punctuation
        
        for char in data[i]:
            if char not in string.punctuation:
                sentence_no_punct = sentence_no_punct + char
        data[i] = sentence_no_punct

        word_tokens = word_tokenize(data[i])
    
        # remove stop words and lemmatize
        word_tokens = [lemmatizer.lemmatize(word) for word in word_tokens if word not in stop_words and len(word) > 1]
        word_tokens = [lemmatizer.lemmatize(word, 'v') for word in word_tokens]
        word_tokens = [lemmatizer.lemmatize(word, 'a') for word in word_tokens]
        
        # remove conjunction words
        word_tokens = [word for word in word_tokens if word[-2:] != 'nt']
        (data[i]) = ' '.join(word_tokens)
        
    return data

In [102]:
def get_flat_descriptions_from_folder(folder):
    descriptions = []
    for i in range(10000):
        with open(('cs5785-fall19-final/descriptions_train/{}.txt').format(i), newline='') as f:
            desc1 = []
            reader = csv.reader(f)
            for row in reader:
                desc1.append(row)
            descriptions.append(desc1)
    
    # FIRST FLATTENING
    descriptions2 = []
    for description in descriptions:
        descriptions2.append([desc for sublist in description for desc in sublist])
        
    # MADE THE SENTENCES ALL ONE FOR EACH DESCRIPTION FILE
    flat_descriptions = []
    for description in descriptions2:
        desc1 = []
        for sentence in description:
            desc1 += sentence.split(' ')
        flat_descriptions.append(desc1)
    
    for i in range(len(flat_descriptions)):
        flat_descriptions[i] = (' ').join(flat_descriptions[i]).lower()
    
    return flat_descriptions

In [103]:
flat_descs_train = get_flat_descriptions_from_folder(descTrainFolder)
flat_descs_test = get_flat_descriptions_from_folder(descTestFolder)

In [104]:
#train_descs = [' '.join(set(preprocessing(desc).split())) for desc in flat_descs_train]
#test_descs = [' '.join(set(preprocessing(desc).split())) for desc in flat_descs_test]

# preprocess
train_descs = preprocessing(flat_descs_train)
test_descs = preprocessing(flat_descs_train)

In [105]:
# training data
resTrainFile = featTrainFolder + "/features_resnet1000_train.csv"
train_feat = pd.read_csv(resTrainFile, header = None, index_col = None)

# testing data
resTrainFile = featTestFolder + "/features_resnet1000_test.csv"
test_feat = pd.read_csv(resTrainFile, header = None, index_col = None)

In [106]:
# sort
for i in range(len(train_feat[0])):
    train_feat[0][i] = int(train_feat[0][i].replace("images_train/", "").replace(".jpg", ""))

train_feat_sort = train_feat.sort_values(by=0)

for i in range(len(test_feat[0])):
    test_feat[0][i] = int(test_feat[0][i].replace("images_test/", "").replace(".jpg", ""))

test_feat_sort = test_feat.sort_values(by=0)

C:\Users\evatr\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\evatr\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [43]:
# CREATE THE BAG OF WORDS DICTIONARY
BOW = {}
for description in train_descs:
    sentence_lst = description.split(' ')
    for word in sentence_lst:
        BOW[word] = 0
BOW['null'] = 0

In [47]:
# CREATE FEATURE VECTORS for TRAIN
feature_vectors = []
for description in train_descs:
    feat_vec = BOW.copy()
    sentence_lst = description.split(' ')
    for word in sentence_lst:
        if word in feat_vec:
            feat_vec[word] += 1
        else:
            feat_vec['null'] += 1       
    feature_vectors.append(feat_vec)
    
# TURN DICTIONARIES INTO A MATRIX with each row as one description
feature_vector_matrix = []
for feature_vec in feature_vectors:
    feature_vector_matrix.append(list(feature_vec.values()))
    
# NORMALIZE THE FEATURES
feature_vector_matrix = sklearn.preprocessing.normalize(feature_vector_matrix) # default is L2 norm

In [ ]:
# CREATE FEATURE VECTORS for TEST
feature_vectors = []
for description in test_descs:
    feat_vec = BOW.copy()
    sentence_lst = description.split(' ')
    for word in sentence_lst:
        if word in feat_vec:
            feat_vec[word] += 1
        else:
            feat_vec['null'] += 1       
    feature_vectors.append(feat_vec)
    
# TURN DICTIONARIES INTO A MATRIX with each row as one description
feature_vector_matrix_test = []
for feature_vec in feature_vectors:
    feature_vector_matrix_test.append(list(feature_vec.values()))
    
# NORMALIZE THE FEATURES
feature_vector_matrix_test = sklearn.preprocessing.normalize(feature_vector_matrix_test) # default is L2 norm

In [4]:
#print(feature_vector_matrix.shape)
#print(train_feat_no_label.shape)
#train_feat_no_label

In [86]:
#test_feat
#train_feat_no_label = train_feat_sort.drop([0], axis=1)
#test_feat_no_label = test_feat_sort.drop([0], axis=1)

In [28]:
# method to transform non-numerical labels (as long as they are hashable and comparable) to numerical labels.
#le = preprocessing.LabelEncoder()
#le.fit(train_descs)
#encoding = le.transform(train_descs)
#print(le.inverse_transform(encoding))
#print(encoding)

['skateboard use man person picnic top skate table crowd pull watch stage show rid boarder put trick skateboarder'
 'enjoy next someone soup noodle bowl ready serve healthy food asian eat sit chopstick tasty carrot ramen shrimp'
 'man intersection cross drive bus ice icecream concession near truck cream street walk across behind busy'
 ...
 'lamp background bedroom cover small nightstand behind room stand next table sit bunch night wall quilt type paper bed'
 'track station drive metal next wall rail train past silver passenger hour travel sit daytime across subway'
 'man board wave surf huge top big surfboard hit shirt rid white trick wipe fall surfer water']
[7770 1631 4256 ... 3195 9115 4176]


In [3]:
#lr_acc, cfn_matrix_lr = Logistic_Regression(feature_vector_matrix, train_feat_no_label, feature_vector_matrix_test, test_feat_no_label)
#gnb_acc, cfn_matrix_gnb = Gaussian_NB(train_descs, train_feat, test_descs, test_feat)
#rf_acc, cfn_matrix_rf = Random_Forest(train_descs, train_feat, test_descs, test_feat)

In [ ]:
#train_feat.drop(train_feat.columns[0], axis=1)
#test = ((((train_feat.loc[:,0]).split('/'))[1]).split('.'))[0]
#test = train_feat.loc[:,0]
#test_2 = (((test.split('/'))[1]).split('.'))[0]
#train_feat[0]

In [1]:
# function to create output csv
def output_csv(neighbors):
    neighbors_adjusted = []
    file_list = []
    for i, row in enumerate(neighbors):
        newlst = []
        for val in row:
            newlst.append(str(val)+'.jpg')
        neighbors_adjusted.append((' ').join(newlst))
        file_list.append(str(i)+'.txt')
    df = pd.DataFrame(zip(file_list, neighbors_adjusted), columns = ['Descritpion_ID', 'Top_20_Image_IDs'])
    df.to_csv('MLP_neuralnet.csv', index = False)